In [1]:
import pandas as pd
import numpy as np
import panel as pn
import geopandas as gpd
pn.extension('tabulator')
import matplotlib.pyplot as plt
from bokeh.models import GeoJSONDataSource, HoverTool
from bokeh.plotting import figure, output_notebook, show

import hvplot.pandas
import holoviews as hv

In [2]:
# load the flood dataset
df = pd.read_excel("FloodArchive.xlsx")

In [3]:
df.head()

,ID,GlideNumber,Country,OtherCountry,long,lat,Area,Began,Ended,Validation,Dead,Displaced,MainCause,Severity
0,1,0,Algeria,0,5.23026,35.81420,92615.67,1985-01-01,1985-01-05,News,26,3000,Heavy rain,1.0
1,2,0,Brazil,0,-45.34890,-18.71110,678498.82,1985-01-15,1985-02-02,News,229,80000,Heavy rain,2.0
2,3,0,Phillipines,0,122.97400,10.02070,12846.03,1985-01-20,1985-01-21,News,43,444,Torrential rain,1.0
3,4,0,Indonesia,0,124.60600,1.01489,16542.12,1985-02-04,1985-02-18,News,21,300,Torrential rain,1.0
4,5,0,Mozambique,0,32.34910,-25.86930,20082.21,1985-02-09,1985-02-11,News,19,0,Heavy rain,2.0


In [4]:
df.shape

(5130, 14)

In [5]:
#Check the data type of the columns
df.dtypes

ID                       int64
GlideNumber             object
Country                 object
OtherCountry            object
long                   float64
lat                    float64
Area                   float64
Began           datetime64[ns]
Ended           datetime64[ns]
Validation              object
Dead                     int64
Displaced                int64
MainCause               object
Severity               float64
dtype: object

In [6]:
df.isna().sum()

ID                 0
GlideNumber     1237
Country            0
OtherCountry    1344
long               0
lat                0
Area               0
Began              0
Ended              0
Validation         0
Dead               0
Displaced          0
MainCause          3
Severity           0
dtype: int64

In [7]:
#check the min and max year to use in the year slider widget
min_year = df['Began'].dt.year.min()
max_year = df['Began'].dt.year.max()

# print the results
print("Minimum year:", min_year)
print("Maximum year:", max_year)

Minimum year: 1985
Maximum year: 2021


In [8]:
# Define Panel widgets 
year_slider = pn.widgets.IntSlider(name='Year slider', start=1985, end=2021, step=2, value=1985)
year_slider

IntSlider(end=2021, name='Year slider', start=1985, step=2, value=1985)

In [9]:
#add total affected columns with is total dead + total displaced
df["TotalAffected"] = df["Dead"] + df["Displaced"]
df['Year'] = df['Began'].dt.year.astype('Int64')

In [10]:
df.head()

,ID,GlideNumber,Country,OtherCountry,long,lat,Area,Began,Ended,Validation,Dead,Displaced,MainCause,Severity,TotalAffected,Year
0,1,0,Algeria,0,5.23026,35.81420,92615.67,1985-01-01,1985-01-05,News,26,3000,Heavy rain,1.0,3026,1985
1,2,0,Brazil,0,-45.34890,-18.71110,678498.82,1985-01-15,1985-02-02,News,229,80000,Heavy rain,2.0,80229,1985
2,3,0,Phillipines,0,122.97400,10.02070,12846.03,1985-01-20,1985-01-21,News,43,444,Torrential rain,1.0,487,1985
3,4,0,Indonesia,0,124.60600,1.01489,16542.12,1985-02-04,1985-02-18,News,21,300,Torrential rain,1.0,321,1985
4,5,0,Mozambique,0,32.34910,-25.86930,20082.21,1985-02-09,1985-02-11,News,19,0,Heavy rain,2.0,19,1985


In [11]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

In [12]:
sorted_df = df.sort_values(by="TotalAffected", ascending=False).head(30)
sorted_df_series = pd.Series(sorted_df["Country"])
sorted_df_series.drop_duplicates(inplace=True)
Countries = np.array(sorted_df_series)
Countries

array(['India', 'Bangladesh', 'Nicaragua\xa0', 'Bangaldesh', ' India',
       'China\xa0', 'Pakistan', 'Nepal', 'China', 'Cambodia',
       'Bangladesh\xa0', 'Thailand', 'Tanzania'], dtype=object)

In [13]:
yaxis= pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Dead', 'Displaced'], 
    button_type='success'
)

dead_displaced_bar_pipeline = (
    idf[
        (idf.Year == year_slider) &
        (idf.Country.isin(Countries))
    ]
    .groupby(['Year', 'Country'])[yaxis].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')  
    .reset_index(drop=True)
)

In [14]:
dead_displaced_bar_pipeline_bar_plot = dead_displaced_bar_pipeline.hvplot(kind='bar', 
                                                     x='Country', 
                                                     y=yaxis, 
                                                     title='Dead or Displaced by Country')
dead_displaced_bar_pipeline_bar_plot

In [15]:
Countries

array(['India', 'Bangladesh', 'Nicaragua\xa0', 'Bangaldesh', ' India',
       'China\xa0', 'Pakistan', 'Nepal', 'China', 'Cambodia',
       'Bangladesh\xa0', 'Thailand', 'Tanzania'], dtype=object)

In [16]:
Test_series = pd.Series(df['Validation'])
Test_series.dropna(inplace=True)
Test_series.drop_duplicates(inplace=True)
Test = np.array(Test_series)

In [17]:
Test

array(['News', 0, 'news', 'Flood List', 'Flood LIst', 'DFO',
       'FloodLIst and GLIDE', 'FloodList', 'FloodList and News',
       'FloodLIst', 'News Reports', 'River Gauging',
       'International Charter', 'Validation', 'NASA-Earth-Observatory',
       'Tropical Storm Track', 'USGS Gauging', 'NASA Flood Alerts',
       'FloodList and other media', 'News Media', 'GFMS Prediction'],
      dtype=object)

In [35]:
year = pn.widgets.IntRangeSlider(name='Years Range Slider', width=250, start=1985, end=2021, value=(1985, 2016), value_throttled=(1985, 2021))

In [36]:
# Map
@pn.depends(year.param.value_throttled)
def plot_map(year):
   years_df = df[df.Began.dt.year.between(year[0], year[1])]
   return df.hvplot.points('long', 'lat', xaxis=None, yaxis=None, geo=True, color='red', alpha=0.5, tiles='CartoLight', hover_cols=["Country", "Dead", "Displaced"], size=5)

In [37]:
# Plot Box
plots_box = pn.WidgetBox(pn.Column(pn.Row(pn.Column(year), pn.layout.HSpacer(), plot_map, align="start", width=600,sizing_mode="stretch_width")))

C:\Users\valter.cheque\Anaconda3\lib\site-packages\cartopy\crs.py:228: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
C:\Users\valter.cheque\Anaconda3\lib\site-packages\cartopy\crs.py:280: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
C:\Users\valter.cheque\Anaconda3\lib\site-packages\cartopy\crs.py:347: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:


In [38]:
dashboard = pn.Row(plots_box, sizing_mode="stretch_width")
dashboard

C:\Users\valter.cheque\Anaconda3\lib\site-packages\cartopy\crs.py:228: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
C:\Users\valter.cheque\Anaconda3\lib\site-packages\cartopy\crs.py:280: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
C:\Users\valter.cheque\Anaconda3\lib\site-packages\cartopy\crs.py:347: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
C:\Users\valter.cheque\Anaconda3\lib\site-packages\cartopy\crs.py:228: Sh

Row(sizing_mode='stretch_width')
    [0] WidgetBox
        [0] Column
            [0] Row(sizing_mode='stretch_width', width=600)
                [0] Column
                    [0] IntRangeSlider(end=2021, name='Years Range Slider', start=1985, value=(1985, 2016), value_end=2016, value_start=1985, width=250)
                [1] HSpacer()
                [2] ParamFunction(function, _pane=HoloViews)